# Intro to df

## Import libs and load dfs

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import tqdm

In [2]:
# Load the datasets

# folder = "PRAMA2025/"
# train_df = pd.read_csv(folder + 'train_with_clusters.csv')
# test_df = pd.read_csv(folder + 'test_with_clusters.csv')

# for those who have the datasets in the same folder as the python code (which is the way to go ngl)
# florian is quite a flexer but we don't have to be like him

train_df = pd.read_csv('train_data.csv')
# test_df = pd.read_csv('test_data.csv')

#little printy print 
print("Train shape: ", train_df.shape)
# print("Test shape: ", test_df.shape)

Train shape:  (17147, 21)


## Find categorical cols

In [27]:
def find_out_col_type(df):
    # Find out the column types
    cat_cols = [col for col in df.columns if df[col].dtype == 'object']
    num_cols = [col for col in df.columns if df[col].dtype in ['int64', 'float64']]
    return cat_cols, num_cols

## Transforming the data

### Attention nouvelle règle : 

- Damien is to come to the next classes with no underwear and a kilt.
- We'll add a value between 0 and 1 showcasing the evolution of the year : 0 for the beginnin of the year, 1 for the end of the year. Then, we'll multiply this by pi and use the sine of this value.
- This way we'll have values closer to 1 for 'colder' days and 0 for 'hotter' days. 

### Sinus of all date data

In [28]:
def sin_ratios_for_date(train_df, test_df):
    # Preprocess the datasets
    # Convert the Sales_Date to datetime format
    train_df['date'] = pd.to_datetime(train_df['date'])
    test_df['date'] = pd.to_datetime(test_df['date'])

    # Extract useful date features (e.g., year, month, day)
    train_df['Year'] = train_df['date'].dt.year
    train_df['Month'] = train_df['date'].dt.month
    train_df['Day'] = train_df['date'].dt.day
    train_df['sin_month'] = train_df['Month'].map(lambda x: np.sin(float(x)/12))

    test_df['Year'] = test_df['date'].dt.year
    test_df['Month'] = test_df['date'].dt.month
    test_df['Day'] = test_df['date'].dt.day
    test_df['sin_month'] = test_df['Month'].map(lambda x: np.sin(float(x)/12))

    train_df['day_of_year'] = train_df['date'].dt.dayofyear
    # Calcul du nombre de jours dans l'année (365 ou 366)
    train_df['days_in_year'] = np.where(train_df['date'].dt.is_leap_year, 366, 365)
    # Calcul El Famoso ratio (float entre 0 et 1)
    train_df['day_ratio'] = train_df['day_of_year'] / train_df['days_in_year']
    # Calculer le sinus de pi * ratio
    train_df['sin_dayofyear'] = np.sin(np.pi * train_df['day_ratio'])
    train_df.drop(columns=['day_of_year', 'days_in_year', 'day_ratio'], inplace=True)
    train_df = train_df.drop(columns=['date', 'year', 'month', 'day'])

    test_df['day_of_year'] = test_df['date'].dt.dayofyear
    # Calcul du nombre de jours dans l'année (365 ou 366)
    test_df['days_in_year'] = np.where(test_df['date'].dt.is_leap_year, 366, 365)
    # Calcul El Famoso ratio (float entre 0 et 1)
    test_df['day_ratio'] = test_df['day_of_year'] / test_df['days_in_year']
    # Calculer le sinus de pi * ratio
    test_df['sin_dayofyear'] = np.sin(np.pi * test_df['day_ratio'])
    test_df.drop(columns=['day_of_year', 'days_in_year', 'day_ratio'], inplace=True)
    test_df = train_df.drop(columns=['date', 'year', 'month', 'day'])

    return train_df, test_df

### Transform df to have price/square meters

In [29]:
def create_new_df_with_m2(train_df, test_df):
    # create a new data frame with everything plus a column corresponding to 1 if "m2_jardin" is >0, 0 otherwise
    # we will also do the same for m2_etage and m2_soussol
    train_new = train_df
    train_new['jardin'] = train_df['m2_jardin'].map(lambda x: 1 if x > 0 else 0)
    train_new['etage'] = train_df['m2_etage'].map(lambda x: 1 if x > 0 else 0)
    train_new['soussol'] = train_df['m2_soussol'].map(lambda x: 1 if x > 0 else 0)

    #do the same for test_df
    test_new = test_df
    test_new['jardin'] = test_df['m2_jardin'].map(lambda x: 1 if x > 0 else 0)
    test_new['etage'] = test_df['m2_etage'].map(lambda x: 1 if x > 0 else 0)
    test_new['soussol'] = test_df['m2_soussol'].map(lambda x: 1 if x > 0 else 0)

    # now add three new columns per df, one for interior m2 and one for exterior m2 and one for total m2
    # for interior we already have m2_interieur which is the sum of etage and soussol
    train_new['m2_outside'] = train_new['m2_jardin']
    test_new['m2_outside'] = test_new['m2_jardin']
    train_new['m2_total'] = train_new['m2_interieur'] + train_new['m2_outside']
    test_new['m2_total'] = test_new['m2_interieur'] + test_new['m2_outside']

    # now we add two columns per df, one for prix per m2 interior and one for prix per m2 total
    train_new['prix_m2_interieur'] = train_new['prix'] / train_new['m2_interieur']
    train_new['prix_m2_total'] = train_new['prix'] / train_new['m2_total']

    # now export
    train_name = "train_with_clusters_and_m2.csv"
    test_name = "test_with_clusters_and_m2.csv"
    train_new.to_csv(train_name, index=False)
    test_new.to_csv(test_name, index=False)

    print("Train shape: ", train_new.shape)
    print("Test shape: ", test_new.shape)

    return train_new, test_new

### Call the transfos

In [3]:
#train_new, test_new = create_new_df_with_m2(train_df, test_df)

# Load the datasets
# folder = "PRAMA2025/"
train_new = pd.read_csv('train_with_clusters_and_m2.csv')
# test_new = pd.read_csv('test_with_clusters_and_m2.csv')

In [4]:
train_new.head()

,id,prix,nb_chambres,nb_sdb,m2_interieur,m2_jardin,m2_etage,m2_soussol,nb_etages,vue_mer,...,Month,Day,cos_month,jardin,etage,soussol,m2_outside,m2_total,prix_m2_interieur,prix_m2_total
0,4443800785,481000,2,1.00,150.501672,360.460795,85.470085,65.031587,1.0,0,...,11,21,0.004426,1,1,1,360.460795,510.962467,3195.977778,941.360727
1,2722059013,550000,2,1.00,117.985879,4046.822742,117.985879,0.000000,1.0,0,...,2,4,-0.416147,1,1,0,4046.822742,4164.808621,4661.574803,132.058889
2,8856004730,199950,2,2.75,147.714604,1943.236715,147.714604,0.000000,1.5,0,...,9,17,-0.911130,1,1,0,1943.236715,2090.951319,1353.623774,95.626330
3,2473003210,364808,3,1.75,215.533259,731.605351,150.501672,65.031587,1.0,0,...,3,13,-0.989992,1,1,1,731.605351,947.138610,1692.583324,385.168545
4,6743700335,470000,3,2.00,167.224080,1176.978818,167.224080,0.000000,1.0,0,...,6,4,0.960170,1,1,0,1176.978818,1344.202899,2810.600000,349.649596


In [ ]:
X_train, y_train, X_test, y_test = train_test_split(train_new.drop(columns=['prix']), train_new['prix'], test_size=0.2, random_state=42)

,id,nb_chambres,nb_sdb,m2_interieur,m2_jardin,m2_etage,m2_soussol,nb_etages,vue_mer,vue_note,...,cluster_tres_eleve,Year,Month,Day,cos_month,jardin,etage,soussol,m2_outside,m2_total
0,3600072,4,2.75,206.243032,493.311037,108.695652,97.547380,1.0,0,0,...,0,2015,3,30,-0.989992,1,1,1,493.311037,699.554069
1,6200017,3,1.00,124.489038,1982.162765,124.489038,0.000000,1.5,0,0,...,0,2014,11,12,0.004426,1,1,0,1982.162765,2106.651802
2,7600136,2,2.00,104.979562,106.651802,74.321813,30.657748,2.0,0,0,...,0,2014,7,18,0.753902,1,1,1,106.651802,211.631364
3,11200400,3,2.50,177.443330,416.945373,177.443330,0.000000,2.0,0,0,...,0,2014,9,23,-0.911130,1,1,0,416.945373,594.388703
4,11500890,3,2.50,290.784095,812.894835,290.784095,0.000000,2.0,0,0,...,0,2015,3,12,-0.989992,1,1,0,812.894835,1103.678930


In [33]:
train_new.drop(columns=['Year', 'Month', 'Day'], inplace=True)
test_new.drop(columns=['Year', 'Month', 'Day'], inplace=True)

clusters = ['cluster_tres_bas', 'cluster_bas', 'cluster_moyen', 'cluster_eleve', 'cluster_tres_eleve']
train_new.drop(columns=clusters, inplace=True)
test_new.drop(columns=clusters, inplace=True)

In [37]:
train_new.columns

Index(['id', 'prix', 'nb_chambres', 'nb_sdb', 'm2_interieur', 'm2_jardin',
       'm2_etage', 'm2_soussol', 'nb_etages', 'vue_mer', 'vue_note',
       'etat_note', 'design_note', 'annee_construction', 'annee_renovation',
       'm2_interieur_15voisins', 'm2_jardin_15voisins', 'zipcode', 'cos_month',
       'jardin', 'etage', 'soussol', 'm2_outside', 'm2_total',
       'prix_m2_interieur', 'prix_m2_total'],
      dtype='object')

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def scaling_data(df1, df2) :
    scaled_name1 = f"train-clusters_sinus_m2_scaled.csv"
    scaled_name2 = f"test-clusters_sinus_m2_scaled.csv"

    # Initialize the MinMaxScaler
    scaler = MinMaxScaler()

    # Scale the data
    to_scale1 = df1  # to not modify the original one
    to_scale2 = df2
    scaled_df1 = pd.DataFrame(scaler.fit_transform(to_scale1), columns=to_scale1.columns)
    scaled_df2 = pd.DataFrame(scaler.transform(to_scale2), columns=to_scale2.columns)

    scaled_df1.to_csv(scaled_name1, index=False)
    scaled_df2.to_csv(scaled_name2, index=False)
    return scaler, scaled_df1, scaled_df2

In [36]:
scaler, train_scaled, test_scaled = scaling_data(train_new, test_new)

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- prix
- prix_m2_interieur
- prix_m2_total
